In [14]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

In [15]:
tf.logging.set_verbosity(tf.logging.INFO)

In [30]:
SAVED_MODEL_PATH = "/mnt/Data/tmp/resnet/plastic_56" # Saved Plastic ResNet checkpoint
TO_SAVE_PATH = SAVED_MODEL_PATH + "/mnist" # Where to save current model
BATCH_SIZE = 32
EPOCHS = 200
LEARN_RATE = 5e-4

In [17]:
def train_input_fn(mnist):
    train_dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,
                                                        mnist.train.labels))

    map_fn = lambda img, lbl: (tf.image.resize_images(tf.reshape(img, [28, 28, 1]),
                                                      (32, 32)), lbl)
    train_dataset = train_dataset.map(map_fn).shuffle(10000).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

    return train_dataset

In [18]:
def test_input_fn(mnist):
    test_dataset = tf.data.Dataset.from_tensor_slices((mnist.test.images,
                                                       mnist.test.labels))

    map_fn = lambda img, lbl: (tf.image.resize_images(tf.reshape(img, [28, 28, 1]),
                                                      (32, 32)), lbl)
    test_dataset = test_dataset.map(map_fn).shuffle(10000).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

    return test_dataset

In [19]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [20]:
def resnet_layer(inputs, eta, scope_name, filters=16, kernel_size=3, strides=1,
                 activation=tf.nn.relu, batch_normalization=True, trainable=False):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    #x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size,
    #                     strides=strides, padding='same')
    with tf.variable_scope(scope_name):
        w = tf.get_variable('conv_w', (kernel_size, kernel_size,
            int(inputs.shape[-1]), filters), trainable=trainable)
        alpha = tf.get_variable('conv_alpha', (kernel_size, kernel_size,
            int(inputs.shape[-1]), filters), trainable=trainable)
        hebb = tf.get_variable('conv_hebb', (kernel_size, kernel_size,
            int(inputs.shape[-1]), filters), trainable=False,
            initializer=tf.zeros_initializer)
        b = tf.get_variable('conv_b', (filters,), trainable=trainable)
    
    x = tf.nn.conv2d(input=inputs, filter=w + tf.multiply(alpha, hebb),
                     strides=[1, strides, strides, 1], padding='SAME') + b
    
    # y is to be the output reshaped so as to be used as a kernel for convolution
    #     on input to get Hebbian update
    y = tf.image.resize_images(x, [int(inputs.shape[1])] * 2)
    y = tf.transpose(y, [1, 2, 0, 3])
    
    # in_mod is the input padded a/c to prev. convolution
    in_mod = tf.pad(inputs, [[0, 0], *([[int(np.floor((kernel_size - 1) / 2)),
            int(np.ceil((kernel_size - 1) / 2))]] * 2), [0, 0]])
    # in_mod is now modded so as to preserve channels and sum over mini-batch
    #     samples for Hebbian update convolution
    in_mod = tf.transpose(in_mod, [3, 1, 2, 0])
    
    hebb_update = tf.nn.conv2d(input=in_mod, filter=y, strides=([1] * 4),
                               padding='VALID')
    hebb = eta * tf.transpose(hebb_update, [1, 2, 0, 3]) + (1 - eta) * hebb
    
    if batch_normalization:
        x = tf.layers.batch_normalization(x, trainable=trainable)
    if activation is not None:
        x = activation(x)
    return x

In [35]:
def resnet_model(features, labels, mode):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    filters = 16
    num_res_blocks = 9
    eta = tf.get_variable('Variable', [1,], trainable=False,
                          initializer=tf.truncated_normal_initializer)
    
    features = tf.cast(features, dtype=tf.float32)
    x = resnet_layer(inputs=features, eta=eta, scope_name='first')
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2
            y = resnet_layer(inputs=x, eta=eta, scope_name="{}_{}_one".format(
                stack, res_block), filters=filters, strides=strides,
                trainable=(stack == 2 and res_block > 4))
            y = resnet_layer(inputs=y, eta=eta, scope_name="{}_{}_two".format(
                stack, res_block), filters=filters, activation=None,
                trainable=(stack == 2 and res_block > 4))
            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, eta=eta, scope_name="{}_{}_three".format(
                    stack, res_block), filters=filters, kernel_size=1,
                    strides=strides, activation=None, batch_normalization=False,
                    trainable=(stack == 2 and res_block > 4))
            x = tf.nn.relu(x + y)
        filters *= 2
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    
    #x = tf.layers.dense(inputs=x, units=10)
    w = tf.get_variable('Variable_1', (int(x.shape[1]), 10),
                        initializer=tf.truncated_normal_initializer)
    alpha = tf.get_variable('Variable_2', (int(x.shape[1]), 10),
                           initializer=tf.truncated_normal_initializer)
    hebb = tf.get_variable('Variable_3', (int(x.shape[1]), 10),
                           trainable=False, initializer=tf.zeros_initializer)
    
    b = tf.get_variable('Variable_4', (10,),
                        initializer=tf.truncated_normal_initializer)
    y = tf.matmul(x, w + tf.multiply(alpha, hebb)) + b
    hebb = eta * tf.reduce_mean(tf.matmul(tf.expand_dims(x, axis=-1),
        tf.expand_dims(y, axis=1)), axis=0) + (1 - eta) * hebb
    x = y
    
    classes = tf.argmax(input=x, axis=1)
    correct_prediction = tf.equal(tf.cast(classes, tf.uint8), labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
        
    onehot_labels = tf.one_hot(tf.cast(labels, tf.int32), 10)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=onehot_labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(
            LEARN_RATE))
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels,
                                                       predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)

In [22]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [23]:
vars_warm = []
for var in ('alpha', 'b', 'hebb', 'w'):
    for stack in range(3):
        for res_block in range(9):
            for layer in ('one', 'two'):
                vars_warm.append("{}_{}_{}/conv_{}".format(stack, res_block, layer,
                                                           var))
            if stack > 0 and res_block == 0:
                vars_warm.append("{}_{}_three/conv_{}".format(stack, res_block,
                                                              var))
    vars_warm.append("first/conv_{}".format(var))
for var in ('beta', 'gamma', 'moving_mean', 'moving_variance'):
    vars_warm.append("batch_normalization/{}".format(var))
    for num in range(1, 55):
        vars_warm.append("batch_normalization_{}/{}".format(num, var))
vars_warm.append("Variable")
#for num in range(1, 5):
    #vars_warm.append("Variable_{}".format(num))
vars_warm += ['beta1_power', 'beta2_power']

In [24]:
warm_start = tf.estimator.WarmStartSettings(
    ckpt_to_initialize_from=SAVED_MODEL_PATH, vars_to_warm_start=vars_warm)

In [37]:
model = tf.estimator.Estimator(model_fn=tf.contrib.estimator.replicate_model_fn(
    resnet_model), model_dir=TO_SAVE_PATH, warm_start_from=warm_start)

INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0'].  Variables are going to be placed on ['/device:GPU:0'].  Consolidation device is going to be /device:GPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/Data/tmp/resnet/plastic_56/mnist', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa4f2109d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [40]:
max_steps = int((mnist.train.images.shape[0] / BATCH_SIZE) * EPOCHS)
model.train(input_fn=lambda:train_input_fn(mnist), hooks=[logging_hook],
            max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/mnt/Data/tmp/resnet/plastic_56', vars_to_warm_start=['0_0_one/conv_alpha', '0_0_two/conv_alpha', '0_1_one/conv_alpha', '0_1_two/conv_alpha', '0_2_one/conv_alpha', '0_2_two/conv_alpha', '0_3_one/conv_alpha', '0_3_two/conv_alpha', '0_4_one/conv_alpha', '0_4_two/conv_alpha', '0_5_one/conv_alpha', '0_5_two/conv_alpha', '0_6_one/conv_alpha', '0_6_two/conv_alpha', '0_7_one/conv_alpha', '0_7_two/conv_alpha', '0_8_one/conv_alpha', '0_8_two/conv_alpha', '1_0_one/conv_alpha', '1_0_two/conv_alpha', '1_0_three/conv_alpha', '1_1_one/conv_alpha', '1_1_two/conv_alpha', '1_2_one/conv_alpha', '1_2_two/conv_alpha', '1_3_one/conv_alpha', '1_3_two/conv_alpha', '1_4_one/conv_alpha', '1_4_two/conv_alpha', '1_5_one/conv_alpha', '1_5_two/conv_alpha', '1_6_one/conv_alpha', '1_6_two/conv_alpha', '1_7_one/conv_alpha', '1_7_two/conv_alpha', '1_8

INFO:tensorflow:Warm-starting from: ('/mnt/Data/tmp/resnet/plastic_56',)
INFO:tensorflow:Warm-starting variable: 0_0_one/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_0_one/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_0_one/conv_alpha
INFO:tensorflow:Warm-starting variable: 0_0_two/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_0_two/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_0_two/conv_alpha
INFO:tensorflow:Warm-starting variable: 0_1_one/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_1_one/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_1_one/conv_alpha
INFO:tensorflow:Warm-starting variable: 0_1_two/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_1_two/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_1_two/conv_alpha
INFO:tensorflow:Warm-starting variable: 0_2_one/conv_alpha; prev_var_na

INFO:tensorflow:Warm-starting variable: 2_0_two/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_0_two/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_0_two/conv_alpha
INFO:tensorflow:Warm-starting variable: 2_0_three/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_0_three/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_0_three/conv_alpha
INFO:tensorflow:Warm-starting variable: 2_1_one/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_1_one/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_1_one/conv_alpha
INFO:tensorflow:Warm-starting variable: 2_1_two/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_1_two/conv_alpha:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_1_two/conv_alpha
INFO:tensorflow:Warm-starting variable: 2_2_one/conv_alpha; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_2_one/conv_alph

INFO:tensorflow:Initialize variable 0_1_one/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_1_one/conv_b
INFO:tensorflow:Warm-starting variable: 0_1_two/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_1_two/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_1_two/conv_b
INFO:tensorflow:Warm-starting variable: 0_2_one/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_2_one/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_2_one/conv_b
INFO:tensorflow:Warm-starting variable: 0_2_two/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_2_two/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_2_two/conv_b
INFO:tensorflow:Warm-starting variable: 0_3_one/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 0_3_one/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 0_3_one/conv_b
INFO:tensorflow:Warm-starting variable: 0_3_two/conv_b; prev_var_name: 

INFO:tensorflow:Warm-starting variable: 2_2_two/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_2_two/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_2_two/conv_b
INFO:tensorflow:Warm-starting variable: 2_3_one/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_3_one/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_3_one/conv_b
INFO:tensorflow:Warm-starting variable: 2_3_two/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_3_two/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_3_two/conv_b
INFO:tensorflow:Warm-starting variable: 2_4_one/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_4_one/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_4_one/conv_b
INFO:tensorflow:Warm-starting variable: 2_4_two/conv_b; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable 2_4_two/conv_b:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with 2_4_t

...

INFO:tensorflow:Warm-starting variable: batch_normalization_52/gamma; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_52/gamma:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_52/gamma
INFO:tensorflow:Warm-starting variable: batch_normalization_52/gamma/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_52/gamma/Adam:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_52/gamma/Adam
INFO:tensorflow:Warm-starting variable: batch_normalization_52/gamma/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_52/gamma/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_52/gamma/Adam_1
INFO:tensorflow:Warm-starting variable: batch_normalization_53/gamma; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_53/gamma:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalizati

INFO:tensorflow:Warm-starting variable: batch_normalization_23/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_23/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_23/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_24/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_24/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_24/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_25/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_25/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_25/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_26/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_26/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/pla

INFO:tensorflow:Initialize variable batch_normalization_54/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_54/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_1/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_1/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_1/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_2/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_2/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_2/moving_variance
INFO:tensorflow:Warm-starting va

INFO:tensorflow:Warm-starting variable: batch_normalization_30/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_30/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_30/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_31/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_31/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_31/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_32/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_32/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with batch_normalization_32/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_33/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_33/moving_varianc

INFO:tensorflow:Warm-starting variable: Variable_2/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable Variable_2/Adam:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with Variable_2/Adam
INFO:tensorflow:Warm-starting variable: Variable_2/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable Variable_2/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with Variable_2/Adam_1
INFO:tensorflow:Warm-starting variable: Variable_4/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable Variable_4/Adam:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with Variable_4/Adam
INFO:tensorflow:Warm-starting variable: Variable_4/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable Variable_4/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/plastic_56 with Variable_4/Adam_1
INFO:tensorflow:Warm-starting variable: beta1_power; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable beta1_power:0 from checkpoint /mnt/Data/tmp/resnet/pla

In [41]:
print(model.evaluate(input_fn=lambda:test_input_fn(mnist)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-18-12:00:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /mnt/Data/tmp/resnet/plastic_56/mnist/model.ckpt-1718
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-18-12:01:07
INFO:tensorflow:Saving dict for global step 1718: accuracy = 0.8895232, global_step = 1718, loss = 0.3564906
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1718: /mnt/Data/tmp/resnet/plastic_56/mnist/model.ckpt-1718
{'accuracy': 0.8895232, 'loss': 0.3564906, 'global_step': 1718}
